In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
#Pull the data from the website
def get_dfs():
    url = "https://data.boston.gov/dataset/property-assessment"
    data = requests.get(url)
    dfs = {}
    parser = BeautifulSoup(data.text,"html.parser")
    year = 2024
    for link in parser.find_all("a",href = True):
        if ".csv" in link['href'] or ".txt" in link["href"]:
            dfs[year] = pd.read_csv(link['href'])
            year -= 1
    return dfs
dfs = get_dfs()




C:\Users\laser\AppData\Local\Temp\ipykernel_96388\3964560085.py:10: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[year] = pd.read_csv(link['href'])
C:\Users\laser\AppData\Local\Temp\ipykernel_96388\3964560085.py:10: DtypeWarning: Columns (6,15,27,28,29,34,35,36,37,38,39,40,41,42,44,45,46,47,48,58,59,64,65,66,67,68,69,70,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[year] = pd.read_csv(link['href'])
C:\Users\laser\AppData\Local\Temp\ipykernel_96388\3964560085.py:10: DtypeWarning: Columns (15,60) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[year] = pd.read_csv(link['href'])
C:\Users\laser\AppData\Local\Temp\ipykernel_96388\3964560085.py:10: DtypeWarning: Columns (15,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[year] = pd.read_csv(link['href'])
C:\Users\laser\AppData\Local\Temp\ipykernel_96388\3964560085.py:10

In [3]:
#Organize Data
for key in dfs:
    dfs[key]["Year"] = key
df_2004_2024 = pd.DataFrame()
for key in dfs:
    dfs[key].rename(columns = {'OWNER MAIL ADDRESS' : 'MAIL_STREET_ADDRESS',"HEAT_FUEL" : "HEAT_SYSTEM", "KITCHEN" : "KITCHENS","EXT_FNISHED":"EXT_FINISHED",\
                     'MAIL_STREET_ADDRESS': 'MAIL_ADDRESS', 'MAIL_ZIP_CODE':'MAIL_ZIPCODE','ZIP_CODE':'ZIPCODE','YR_REMOD':'YR_REMODEL','OWNER_MAIL_ZIPCODE' : 'MAIL_ZIPCODE','OWNER_MAIL_CS' : 'MAIL CS',\
                              'OWNER_MAIL_ADDRESS':'MAIL_ADDRESS','Owner_MAIL_ZIPCODE':'MAIL_ZIPCODE','Owner_MAIL_CS':'MAIL CS','Parcel_ID':'PID','Owner_MAIL_ADDRESS':'MAIL_ADDRESS',\
                              'FY2008_BLDG' : 'BLDG_TYPE', 'FY2008_GROSS_TAX' : 'GROSS_TAX','FY2008_LAND' : 'LAND_VALUE', 'FY2008_TOTAL' : 'TOTAL_VALUE',\
                               'AV_LAND' :'BLDG_VALUE' , 'AV_TOTAL' : 'TOTAL_VALUE'},inplace = True)
dfs[2005]["ZIPCODE"] = dfs[2005]["PTYPE"]
df_2004_2024 = pd.concat(dfs)
df_2004_2024 = df_2004_2024[["Year","ZIPCODE"]]

In [4]:
#Clean Zipcodes
def zipcodefun(x):
    x= str(x)
    if x in ['_','0NULL_','000D_','00000_', "3.0","1","9R","1.0","8.0",'00420_',"nan","00003","0000D"\
            "00405","00403","00105"," '098221","0None"]:
        return None
    if ".0" in x:
        x = x.replace(".0","")
        x = "0" + x
        return x
    if x[-1] == "_":
        x = x[0:-1]
        return x
    if len(x) == 4:
        if x[0] == "0":
            return x + "0"
        return "0" + x
    return x
        
    
df_2004_2024["ZIPCODE"] = df_2004_2024["ZIPCODE"].apply(lambda x : zipcodefun(x))
df_2004_2024.to_csv("property_assessments.csv",index = False)

In [5]:
zipdict = {'02128' : "EAST BOSTON", '02115' : "KENMORE", '02130' : "JAMAICA PLAIN", '02129' : "CHARLESTOWN" ,\
 '02114' :"DOWNTOWN",  '02113' : "NORTH END", '02109' : "NORTH END", '02108' : "BEACON HILL" , '02110' : "DOWNTOWN",\
 '02201' : "DOWNTOWN", '02210' : "SOUTH BOSTON" ,'02116' : "BACK BAY" , '02111' : "CHINATOWN", '02127' : "SOUTH BOSTON",\
 '02118' : "SOUTH END",'02215' : "KENMORE", '02120' : "JAMAICA PLAIN",\
 '02125' : "HARBOR ISLANDS", '02122' : "DORCHESTER", '02124' : "DORCHESTER", '02131' : "ROSINDALE", '02121' : "ROXBURY",\
 '02126' : "MATTAPAN", '02136' : "HYDE PARK", '02135' : "ALLSTON", '02132' : "WEST ROXBURY",
 '02467' : "CHESTNUT HILL", '02134' : "ALLSTON", '02167' : "CHESTNUT HILL"}
df_2004_2024["ZIPCODE"] = df_2004_2024["ZIPCODE"].apply(lambda x : x if x in zipdict else None)
df_2004_2024 = df_2004_2024[df_2004_2024["ZIPCODE"].notna()]
df_2004_2024["NEIGHBORHOOD"] = df_2004_2024["ZIPCODE"].apply(lambda x : zipdict[x])

In [7]:
dfpercent = pd.DataFrame(columns = ["Year","NEIGHBORHOOD","change","%change","count"])
nums = df_2004_2024.groupby("NEIGHBORHOOD")["Year"].value_counts()
for nh in df_2004_2024["NEIGHBORHOOD"].unique():
    for year in range(2004,2025):
        if year != 2004:
            p = nums[nh][year] / nums[nh][year-1]
            p = (p-1) *100
            c = nums[nh][year] - nums[nh][year-1]
            dfpercent.loc[len(dfpercent)+1] = {"Year":year,"NEIGHBORHOOD":nh,"change" : c,"%change":p,"count":nums[nh][year]}
        else:
            dfpercent.loc[len(dfpercent)+1] = {"Year":year,"NEIGHBORHOOD":nh,"change" : None, "%change": None,"count":nums[nh][year]}
    
            
            
        

In [8]:
num_counts = df_2004_2024.groupby("Year")
for year in range(2005,2025):
    if year != 2004:
        count = np.sum(num_counts.value_counts()[year])
        prev_count = np.sum(num_counts.value_counts()[year-1])
        c = count - prev_count
        p = count / prev_count
        p = (p-1) *100
        dfpercent.loc[len(dfpercent)+1] = {"Year":year,"NEIGHBORHOOD": "BOSTON TOTAL","change" : c,"%change":p,"count":count}
    else:
        count = np.sum(num_counts.value_counts()[year])
        dfpercent.loc[len(dfpercent)+1] = {"Year":year,"NEIGHBORHOOD": "BOSTON TOTAL","change" : None,"%change":None,"count":count}
        
        
    

In [84]:
dfpercent.to_csv("neighborhood_changes_2004_2024.csv",index = False)